**Analysis of Variance - ANOVA**

**Error Types**
> Type I and Type II errors are statistical concepts that describe the errors that can occur when making decisions based on statistical tests. They are important because they help us to understand the trade-offs between the risks of making incorrect decisions and the benefits of making correct decisions.
>
> Balancing these two errors is important in hypothesis testing. Usually, reducing the risk of one type increases the risk of the other, so trade-offs are made based on the context and importance of each type of error.
> 
>  <img src="images/Error Types.png" alt="Error Types" width="400" style="display: block; margin: 0 auto;"/>

**Family-Wise Error Rate (FWER)**
> Family-wise error rate (FWER) refers to the probability of making at least one Type I error (false positive) when conducting multiple statistical tests. In simpler terms, it addresses the concern that when multiple hypotheses are tested simultaneously, the chances of incorrectly rejecting at least one null hypothesis increase.
>

**ANOVA Tests**
> ANOVA is a statistical method used to compare the means of three or more groups to determine if there are statistically significant differences between them. It tests the hypothesis that the group means are equal by analyzing the variance within each group and between groups.
>
> ANOVA cannot tell which specific groups were statistically significantly different from each other, only that at least two groups were. To determine which specific groups differed from each other, we need to use a post-hoc test.
> 
**1. One-Way ANOVA** - Single Factorial
> One-way ANOVA is used to determine whether there are any statistically significant differences between the means of three or more independent (unrelated) groups based on a single factor or independent variable. It is an extension of the t-test for independent samples to more than two groups.
> |  |  |  |
> |--|--|--|
> | $H_0$ | $μ_1 = μ_2 =... = μ_n$                          | → There are no difference between means of the groups. |
> | $H_1$ | $∃i,j \ \ such\ that \ \ i ≠ j\ and\ μ_i ≠ μ_j$ | → At least one group with a different mean. |
>
> |  |  |
> |--|--|
> |Single Factorial without meausrement repetition |→ Groups are independent |
> |Single Factorial with meausrement repetition    |→ Same group
>
> **ASSUMTIONS**
>
>  → Data within the groups should be normally distributed. (Residuals should be normally distributed.)<br>
>  → The Variance of data in groups should be equal. <br>
> > If homogeneity assumption is not met, we should use **Welch's ANOVA** which is an alternative to the classic ANOVA and can be used even if your data violates the assumption of homogeneity of variances.
> 
> → The measurements should be independent. <br>
> → The dependent variable should have a metric scale level.

**Post-hoc Test**
> When there is a significant difference between groups, various statistical tests are used to determine which groups are different from each other. These tests determine which groups are statistically different from each other by comparing all combinations between the groups.
> > | Post-Hoc Test | Equal Variance | Equal Sample Size |
> |-|:--:|:--:|
> | Bonferroni |✔ | ✗ |
> | Tukey HSD | ✔ | ✔ |
> | Scheffe | ✔ | ✗ |
> | Tamhane’s T2 | ✗ | ✗ |

<p style="background-image: linear-gradient(to right, #0aa98f, #68dab2)"> &nbsp; </p>

In [1]:
import pandas as pd
import pingouin as pg
from scipy import stats
import statsmodels.stats as ss
import scikit_posthocs as sp

<p style="background-image: linear-gradient(#0aa98f, #ffffff 10%); font-weight:bold;"> 
    &nbsp; Functions to Use </p>

In [2]:
α = alpha = 0.05

def decision(p, alpha=0.05):
    'acceptance or rejection of the null hypothesis'
    if p < alpha: return 'H0 rejected.'
    else: return 'H0 cannot be rejected.'

def decision_table(p, alpha=0.05):
    'acceptance or rejection of the null hypothesis for table representation'
    if p < alpha: return '✗'
    else: return '✔'

<p style="background-image: linear-gradient(to right, #0aa98f, #68dab2)"> &nbsp; </p>

<p style="background-image: linear-gradient(#0aa98f, #ffffff 10%); font-weight:bold;"> 
    &nbsp; ONE-WAY ANOVA </p>
    
**Subject :** Comparison of television viewing time according to educational status <br>
**Data :** 5_tv_view.csv

|  |  |  |
|--|--|--|
| $H_0$ | $μ_1 = μ_2 =... = μ_n$                          | → There is no difference in average television viewing time between groups according to educational status. |
| $H_1$ | $∃i,j \ \ such\ that \ \ i ≠ j\ and\ μ_i ≠ μ_j$ | → According to educational attainment, at least one group differs from the others in terms of average television viewing time. |

In [3]:
data = pd.read_csv('data/05_tv_view.csv', index_col='Respondent')
display(data.sample(3))

,Education Level,TV Watch Time
Respondent,,
12,Bachelor,70
11,Bachelor,73
20,Master,55


**1. Data Preparation**

In [4]:
groups = data.groupby('Education Level')['TV Watch Time']
_groups = [group for _, group in groups] # just groups without names

# for name, group in groups:
#     print(name)
#     print(group.to_list())
# # print(*groups, sep='\n')

**2. Normality Test**

In [5]:
# for name, group in groups:
#     normality = stats.shapiro(group).pvalue
#     print(f'{name.ljust(10)} p: {normality:.4f} \t Decision: {decision(normality)}')

normality = pg.normality(data, dv='TV Watch Time', group='Education Level', method='shapiro', alpha=alpha)
display(normality)

,W,pval,normal
Education Level,,,
Primary,0.905493,0.440988,True
High,0.887885,0.346581,True
Bachelor,0.898244,0.400257,True
Master,0.839882,0.164596,True


**3. Homogeneity Test**

In [6]:
# homogeneity = stats.bartlett(*_groups).pvalue
# print(f'p: {homogeneity:.4f} \t Decision: {decision(homogeneity)}')

homogeneity = pg.homoscedasticity(data, dv='TV Watch Time', group='Education Level', method='bartlett')
display(homogeneity)

,T,pval,equal_var
bartlett,0.428628,0.934267,True


**4. Test Implementation** - One-Way ANOVA

In [7]:
# F, p = stats.f_oneway(*_groups)

test = pg.anova(data=data, dv='TV Watch Time', between='Education Level')
F, p = test.loc[0, ['F', 'p-unc']]

print(f'F: {F:.4f}')
print(f'p: {p:.4f}')
print('Decision:', decision(p, α))

F: 5.8720
p: 0.0067
Decision: H0 rejected.


**5. Research of Different Groups** - Post-Hoc / Tukey HSD Test

**There are differences between the groups that have a 'True' value in the 'reject' column*

In [8]:
# tukey_hsd = ss.multicomp.pairwise_tukeyhsd(data['TV Watch Time'], data['Education Level'], alpha=alpha)
# print(tukey_hsd)


# tukey_hsd = sp.posthoc_tukey_hsd(data['TV Watch Time'], data['Education Level'])
# for i in range(tukey_hsd.shape[0]):
#     tukey_hsd.iloc[i,i] = 0
# display(tukey_hsd)


tukey_hsd = pg.pairwise_tukey(data=data, dv='TV Watch Time', between='Education Level')
tukey_hsd['Reject'] = tukey_hsd['p-tukey'].map(lambda p: False if p>alpha else True)
# tukey_hsd['Decision'] = tukey_hsd['p-tukey'].map(decision_table)
display(tukey_hsd)
display(tukey_hsd[tukey_hsd['Reject']==True])

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges,Reject
0,Bachelor,High,72.0,62.0,10.0,5.495453,1.819686,0.300470,1.137958,False
1,Bachelor,Master,72.0,53.0,19.0,5.495453,3.457404,0.015411,1.872450,True
2,Bachelor,Primary,72.0,73.0,-1.0,5.495453,-0.181969,0.997769,-0.099442,False
3,High,Master,62.0,53.0,9.0,5.495453,1.637718,0.386712,0.982186,False
4,High,Primary,62.0,73.0,-11.0,5.495453,-2.001655,0.228254,-1.213813,False
5,Master,Primary,53.0,73.0,-20.0,5.495453,-3.639373,0.010667,-1.925684,True


,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges,Reject
1,Bachelor,Master,72.0,53.0,19.0,5.495453,3.457404,0.015411,1.872450,True
5,Master,Primary,53.0,73.0,-20.0,5.495453,-3.639373,0.010667,-1.925684,True


**6. Display of Different Groups**

In [9]:
display(groups.mean().to_frame().T)

# display(
#     groups.mean()[['Bachelor', 'Master']],
#     groups.mean()[['Master', 'Primary']] )

Education Level,Bachelor,High,Master,Primary
TV Watch Time,72.0,62.0,53.0,73.0


<p style="background-image: linear-gradient(to right, #ee2965, #e31837)"> &nbsp; </p>

**4.2 If the assumption of homogeneity of variances is violated**

In [10]:
welch_test = pg.welch_anova(data=data, dv='TV Watch Time', between='Education Level')
F, p = welch_test.loc[0, ['F', 'p-unc']]

# print(welch_test)
print(f'F: {F:.4f}')
print(f'p: {p:.4f}')
print('Decision:', decision(p, α))

F: 4.5717
p: 0.0338
Decision: H0 rejected.


**5.2 Research of Different Groups** - Post-Hoc / Tamhane's T2 Test

In [11]:
tamhane = sp.posthoc_tamhane(data, val_col='TV Watch Time', group_col='Education Level')
tamhane = tamhane.map(decision_table)
display(tamhane)

,Primary,High,Bachelor,Master
Primary,✔,✔,✔,✗
High,✔,✔,✔,✔
Bachelor,✔,✔,✔,✗
Master,✗,✔,✗,✔


<p style="background-image: linear-gradient(to right, #0aa98f, #68dab2)"> &nbsp; </p>